In [1]:
pip install --upgrade pandas

Requirement already up-to-date: pandas in /Users/owner/opt/anaconda3/lib/python3.7/site-packages (1.0.4)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

In [4]:
dataframe1 = pd.read_csv('static/Train_test_dataset.csv')
dataframe = dataframe1.drop(["Unnamed: 0"], axis=1)
dataframe.head()

,label,eligibility_criteria,interventionname
0,0,fulfilling all the following criteria,Hydroxychloroquine
1,0,fulfilling all the following criteria,Hydroxychloroquine
2,0,"aged between 18 and 60 years, male or female.",Hydroxychloroquine
3,0,"aged between 18 and 60 years, male or female.",Hydroxychloroquine
4,0,hospitalized participants with diagnosis of co...,Hydroxychloroquine


In [5]:
train, test = train_test_split(dataframe, test_size=0.2)

print(len(train), 'train examples')

print(len(test), 'test examples')

1956 train examples
490 test examples


In [21]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  return ds

In [22]:
# batch_size = 5 # A small batch sized is used for demonstration purposes
# train_ds = df_to_dataset(train, batch_size=batch_size)

# test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [23]:
# for feature_batch, label_batch in train_ds.take(1):
#   print('Every feature:', list(feature_batch.keys()))
  

In [24]:
# We will use this batch to demonstrate several types of feature columns
# example_batch = next(iter(train_ds))[0]

In [25]:
# A utility method to create a feature column
# and to transform a batch of data
# def demo(feature_column):
#   feature_layer = layers.DenseFeatures(feature_column)
#   print(feature_layer(example_batch).numpy())

In [26]:
batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)

test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [27]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['eligibility_criteria', 'interventionname']


In [41]:
# Notice the input to the embedding column is the categorical column
# we previously created
feature_columns = []
eligibility_criteria_embedding = feature_column.embedding_column(dataframe["eligibility_criteria"], dimension=8)
feature_columns.append(eligibility_criteria_embedding)
eligibility_criteria_embedding

EmbeddingColumn(categorical_column=0                   fulfilling all the following criteria
1                   fulfilling all the following criteria
2           aged between 18 and 60 years, male or female.
3           aged between 18 and 60 years, male or female.
4       hospitalized participants with diagnosis of co...
                              ...                        
2441                        calcium <8.4mg/dl >10.6mg/dl*
2442                        calcium <8.4mg/dl >10.6mg/dl*
2443                           potassium <3.3 >5.5 meg/l*
2444                           potassium <3.3 >5.5 meg/l*
2445                           potassium <3.3 >5.5 meg/l*
Name: eligibility_criteria, Length: 2446, dtype: object, dimension=8, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x137b67150>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

In [35]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [36]:
from tensorflow.keras.models import Sequential

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_ds, epochs=10)

AttributeError: 'Series' object has no attribute '_num_buckets'